# Location of dataset

https://www.reddit.com/r/datasets/comments/1uyd0t/200000_jeopardy_questions_in_a_json_file


### Description:
  The json file is an unordered list of questions where each question has:
  
- 'category' : the question category, e.g. "HISTORY"
- 'value' : dollar value of the question as string, e.g. "\$200". $Note$: This is "None" for Final Jeopardy! and Tiebreaker questions
- 'question' : text of question. $Note$: This sometimes contains hyperlinks and other things messy text such as when there's a picture or video question
- 'answer' : text of answer
- 'round' : one of "Jeopardy!","Double Jeopardy!","Final Jeopardy!" or "Tiebreaker" Note: Tiebreaker questions do happen but they're very rare (like once every 20 years)
- 'show_number' : string of show number, e.g '4680'
- 'air_date' : the show air date in format YYYY-MM-DD


In [23]:
import pandas as pd
import numpy as  np
import string , re

pd.options.display.max_rows = 6
pd.options.display.max_columns = 6
pd.options.display.width = 80

In [33]:
#jeopardy = pd.read_json('JEOPARDY_QUESTIONS1.json')
jeopardy = pd.read_csv('JEOPARDY_CSV.csv')

In [34]:
jeopardy.sample(10)

,Show Number,Air Date,Round,...,Value,Question,Answer
43452,4965,2006-03-24,Jeopardy!,...,$400,"1933: <a href=""http://www.j-archive.com/media/...",King Kong
63162,967,1988-11-15,Jeopardy!,...,$500,A 1979 hit by Supertramp gave you these direct...,"""Take The Long Way Home"""
119820,3416,1999-06-14,Jeopardy!,...,$300,"Hey, steer clear of the fellow seen here (skun...",Polecat
...,...,...,...,...,...,...,...
45247,3172,1998-05-19,Jeopardy!,...,$500,This English scientist didn't need a cat to fa...,Sir Isaac Newton
204127,11,1984-09-24,Double Jeopardy!,...,$400,"Richard Carlson ""led"" this number of lives as ...",3
70645,5169,2007-02-15,Double Jeopardy!,...,$1200,The song mentions their pride,Pilgrims


In [38]:
jeopardy.columns = jeopardy.columns.str.strip().str.replace(' ','_').str.lower()

In [39]:
new_names = [i.lower().strip().replace(' ','_') for i in jeopardy.columns.tolist()]

In [40]:
jeopardy.columns = new_names

In [32]:
jeopardy.sample(2)

,show_number,air_date,round,...,clean_question,clean_answer,clean_value
193932,4770,2005-05-06,Jeopardy!,...,to write down words especially from audio to w...,transcribe,400
108239,2949,1997-05-29,Double Jeopardy!,...,limousin blonde daquitaine are popular breeds...,france,400


In [ ]:
jeopardy

### Performing basic cleanup operations:

- Stripping all punctuation and capitalizations in the 'Question' and 'Answer' columns.
- Converting the 'Value' column to integer type after stripping dollar sign.
- Converting the 'Air_date' column values to datetime objects.

Refer to these links:

http://www.tutorialspoint.com/python3/string_maketrans.htm

https://stackoverflow.com/questions/11692199/string-translate-with-unicode-data-in-python

https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python


Alternate Method:
```python
print(string.punctuation)

def clean(obj):
    punctuation = string.punctuation
    remove_uni_ordinal_mapping = dict((ord(char),None) for char in punctuation)
    obj = obj.translate(remove_uni_ordinal_mapping)
    
    return obj.lower()
```

In [29]:
def clean(obj):
    tbl = string.maketrans("","")
    obj = obj.translate(tbl,string.punctuation).lower()
    return obj

def str_2_num(obj):
    try:
        return np.int(obj)
    except:
        return 0

In [30]:
jeopardy['clean_question'] = jeopardy.question.apply(clean)
jeopardy['clean_answer'] = jeopardy.answer.apply(clean)
jeopardy['clean_value'] = jeopardy.value.apply(clean)
jeopardy.clean_value = jeopardy.clean_value.apply(str_2_num)
jeopardy.air_date = pd.to_datetime(jeopardy.air_date, format='%Y-%m-%d ')

In [31]:
jeopardy.sample(2)

,show_number,air_date,round,...,clean_question,clean_answer,clean_value
1927,6294,2012-01-19,Jeopardy!,...,the chorus of romeo juliet tells us its in th...,verona,600
39123,5032,2006-06-27,Double Jeopardy!,...,there are 2 kinds of this creature soft hard ...,ticks,800


In [19]:
(jeopardy[['question','answer']]
                                 #.question
                                 #.apply(clean)
                                 .answer
                                 .apply(clean)
)

0                                                copernicus
1                                                jim thorpe
2                                                   arizona
3                                                 mcdonalds
4                                                john adams
5                                                   the ant
6                                            the appian way
7                                            michael jordan
8                                                washington
9                                             crate  barrel
10                                           jackie gleason
11                                                  the cud
12                                      ceylon or sri lanka
13                                                jim brown
14                                             the uv index
15                                                   bulova
16                                      

In [ ]:
def splitS(Series):
    try:
        return Series.str.split(" ")
    except:
        return 0

def removeS(Series,rm_string):
    try:
        for k,v in Series.iteritems():
            v.remove(rm_string)
        return 0
    except:
        return 1

In [ ]:
jeopardy['split_answer'] = splitS(jeopardy.clean_answer)
jeopardy['split_question'] = splitS(jeopardy.clean_question)

In [ ]:
#Picking out a 'the' in jeopardy.split_question column of dataframe:
jeopardy.split_question.loc[:3]

In [ ]:
test = removeS(jeopardy.split_question,'the')
answer = removeS(jeopardy.split_answer,'the')

In [ ]:
def matching_function(df):
    match_count = 0
    split_question = splitS(df['clean_question'])
    split_answer = splitS(df['clean_answer'])
    remove_the = removeS(split_question,'the')
    remove_the = removeS(split_answer,'the')
    return split_question,split_answer
    

In [ ]:
X, Y = matching_function(jeopardy[['clean_question','clean_answer']])

In [ ]:
X.loc[0] , Y.loc[0]

In [ ]:
test, answer